In [122]:
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import pandas as pd

In [157]:
def find_related_words(word):
    synonyms = set()
    synonyms.add(word)
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower())

    return synonyms

In [158]:
def delete_punctuation(text):
    # Remove punctuation using the string module
    translator = str.maketrans('', '', string.punctuation)
    text_without_punct = text.translate(translator)
    return text_without_punct

def delete_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    return [word for word in word_tokens if word.lower() not in stop_words]

In [159]:
def keywords_text(text):
    return delete_stopwords(delete_punctuation(text))

In [160]:
df = pd.read_csv('Destinations.csv')

In [161]:
df

,Destinations,country,region,Climate,Budget,Activity,Demographics,Duration,Cuisine,History,Natural Wonder,Accommodation,Language
0,Tokyo,Japan,East Asia,Temperate,High,Cultural,Solo,Long,Asian,Modern,Mountains,Luxury,Japanese
1,Ottawa,Canada,North America,Cold,Medium,Adventure,Family-friendly,Medium,European,Modern,Forests,Mid-range,English
2,Mexico City,Mexico,North America,Temperate,Low,Cultural,Senior,Short,Latin American,Ancient,Mountains,Budget,Spanish
3,Rome,Italy,Southern Europe,Temperate,High,Cultural,Solo,Medium,European,Ancient,Beaches,Luxury,Italian
4,Brasilia,Brazil,South America,Tropical,Low,Adventure,Family-friendly,Long,Latin American,Modern,Beaches,Budget,Portuguese
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Luxor,Egypt,North Africa,Desert,Low,Cultural,Solo,Short,Middle Eastern,Ancient,Rivers,Budget,Arabic
98,Aswan,Egypt,North Africa,Desert,Low,Cultural,Solo,Short,Middle Eastern,Ancient,Rivers,Budget,Arabic
99,Christchurch,New Zealand,Oceania,Temperate,Medium,Adventure,Family-friendly,Medium,Western,Modern,Cityscape,Mid-range,English
100,Queenstown,New Zealand,Oceania,Temperate,High,Adventure,Solo,Short,Western,Modern,Mountains,Luxury,English


In [162]:
def sanatize(string):
    return string.lower().replace('\'', '').replace('.', '').split(',')[0]

In [163]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [164]:
categories = {}
for col in df.columns:
    categories[sanatize(col)] = find_related_words(sanatize(col))
    categories[sanatize(col)].add(sanatize(col))

In [165]:
informations = {}
for col in df.columns:
    informations[sanatize(col)] = {}
    for val in df[col]:
        informations[sanatize(col)][sanatize(val)] = {}

In [166]:
for category in informations:
    for val in informations[category]:
        informations[category][val] = find_related_words(val)

In [167]:
def extract_features(text):
    keywords = keywords_text(text)
    features = {}
    for category in categories:
        for value in categories[category]:
            if value in keywords:
                features[category] = set()  
                
    for category in features:
        for value in informations[category]:
            for related in informations[category][value]:
                if related in keywords:
                    features[category].add(value)

    return features

In [176]:
text = "I want go trip with low budget and finish in tokyo and with gothic story"

In [177]:
print(extract_features(text))

{'destinations': {'tokyo'}, 'budget': {'low'}, 'history': {'medieval'}}
